In [ ]:
import requests
import pandas as pd
# from google.colab import drive
import pickle
import os


# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
day_request = 0

## Caveats

Before running the Notebook, make sure to select the correct user and the csv you want to obtain images from.

Also bare in mind that this Notebook asumes that the 'CameraDetection' folder is located in your "My Drive" section.

Also Adjust the  'request_limit' to a number with which you think you will not exceed the number of requests.



In [ ]:
## Choose User
user_name = 'Claudio'
# user_name = 'Haidar'

# ADJUST LIMIT!!  -- Each month we have 42000 image requests (per account). I have decreased a little the request limit just in case.
account_request_limit = 41900
day_request_limit = 25000      ## Per day   -- 25000 requests       |    Per Month  -- 42000 requests

if user_name == 'Claudio':

    target_csv_name = 'sub_dataset_1.csv'   #  Claudio´s CSVs : sub_dataset_1.csv, sub_dataset_2.csv,sub_dataset_3.csv, sub_dataset_4.csv, sub_dataset_5.csv

    path_to_main_folder = 'path to main folder'
    path_to_image_folder = path_to_main_folder+'/images/C-images/'  + target_csv_name.split('.')[0]
    dataPoints = pd.read_csv(path_to_main_folder+'/data/'+target_csv_name)
    path_to_pickle = path_to_main_folder+'/data/claudio_actual_requests.pickle'

    # Define your API key and signature
    api_key = ''

    # Load the counter from the file
    with open(path_to_pickle, 'rb') as file:
        actual_requests = pickle.load(file)['actual_requests']

    print('User: '+ user_name+ ' | Remaining Requests: '+ str(account_request_limit-actual_requests) + ' | Requests Performed: ', actual_requests)

elif user_name == 'Haidar':

    target_csv_name = 'sub_dataset_6.csv'  #  Haidar´s CSVs : sub_dataset_6.csv, sub_dataset_7.csv,sub_dataset_8.csv, sub_dataset_9.csv, sub_dataset_10.csv

    path_to_main_folder = 'path to main folder'
    path_to_image_folder = path_to_main_folder+'/images/H-images/' + target_csv_name.split('.')[0]
    path_to_pickle = path_to_main_folder+'/data/'
    dataPoints = pd.read_csv(path_to_main_folder+'/data/'+target_csv_name)

    # Define your API key and signature
    api_key = ''
    path_to_pickle = path_to_main_folder+'/data/haidar_actual_requests.pickle'

    with open(path_to_pickle, 'rb') as file:
        actual_requests = pickle.load(file)['actual_requests']


    print('User: '+ user_name+ ' | Remaining Requests: '+ str(account_request_limit-actual_requests) + ' | Requests Performed: ', actual_requests)

else:
    print('WARNING: Wrong User')


User: Claudio | Remaining Requests: 0 | Requests Performed:  41900


Before running the Image Extraction Code make sure that the range of images that the code is going to scrape is the one that you want.

In [ ]:
## In case you are starting with a new dataset
# data_range = range(dataPoints.shape[0])

last_img_id = 39528  # The real id of the image (id column in dataset)

## In case you are continuing with a dataset
data_range = range(dataPoints[dataPoints['id'] > last_img_id].index[0],dataPoints.shape[0])

## In case you want to decide your own data range
# data_range = range(dataPoints[dataPoints['id'] > last_img_id].index[0],10)

print('Scraping Range : ', data_range,  ' CSV file: ',target_csv_name)

Scraping Range :  range(7998, 11360)  CSV file:  sub_dataset_3.csv


-------
# Image Extraction Code

In [ ]:
safety_input = input('Are you sure that you want to Extract Images ? [y/n]')

no_image_count = 0

processed_coords = 0



for image_num in data_range:
    if safety_input == 'y' and actual_requests < account_request_limit and day_request < day_request_limit:

        if processed_coords % 1000 == 0:
          print('Processed Coordinates : ', processed_coords)

        data_point = dataPoints.iloc[image_num]
        size = '640x640'
        fov = '80'
        pitch = '8'
        location = str(data_point['lat']) + ',' + str(data_point['lon'])
        headings = [data_point['heading_forward'] + 90, data_point['heading_forward'] - 90]
        id_ = data_point['id']

        ## Making a Metadata call (with one of the headings) to see if there is an image for that data point
        check_status = f'https://maps.googleapis.com/maps/api/streetview/metadata?size={size}&location={location}&fov={fov}&pitch={pitch}&heading={headings[0]}&key={api_key}'
        response = requests.get(check_status)

        if response.json()['status'] == 'OK':

            for heading in headings:

                # These are actual requests
                api_url = f'https://maps.googleapis.com/maps/api/streetview?size={size}&location={location}&fov={fov}&pitch={pitch}&heading={heading}&key={api_key}'
                response = requests.get(api_url)
                actual_requests += 1
                day_request += 1

                image_name = path_to_image_folder+'/strtV_image_'+str(int(id_))+ '_'+ str(abs(int(heading))) + '.jpg'

                # Check if the request was successful
                if response.status_code == 200:
                    with open(image_name, 'wb') as image_file:
                        image_file.write(response.content)

                else:
                    print(f'Error: Failed to retrieve Street View image. Status code {response.status_code}')

        else:
          no_image_count+=1

        processed_coords += 1

    else:
      print('WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)')
      break
## Saving the Updated number of requests performed
with open(path_to_pickle, 'wb') as file:
    pickle.dump({'actual_requests':actual_requests}, file)


if no_image_count > 0:
  print('\nFor this csv was not possible to obtain an image for ',no_image_count, ' locations')


print('\nThe last image id that has been processed is :',int(id_))

Processed Coordinates :  0
Processed Coordinates :  1000
Processed Coordinates :  2000
Processed Coordinates :  3000
Processed Coordinates :  4000
Processed Coordinates :  5000
Processed Coordinates :  6000
Processed Coordinates :  7000
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded your request limit (or you decided to not run your code hehe :)
WARNING!! You have exceeded

### In case of Google Collab Colapsing

Kill the execution of the ongoing cell an print the following to update the pickles.

In [ ]:
print("Actual Requests: ", actual_requests)
print("Last Image ID: ", int(id_))
print("Nº of non returning locations: ", no_image_count)
print("Day Request : ",day_request)

Actual Requests:  41900
Last Image ID:  39528
Nº of non returning locations:  33
Day Request :  15930


In [ ]:
# with open(path_to_pickle, 'wb') as file:
#     pickle.dump({'actual_requests':actual_requests}, file)